In [34]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

stocks = ['AMD', 'NVDA', 'META', 'TSLA']
start = datetime(2018, 1, 1)
end = datetime(2024, 4, 30)

stock_prices = yf.download(stocks, start, end, auto_adjust=False)
stock_prices = stock_prices.resample('ME').last()
stock_prices.index = stock_prices.index.tz_localize(None)
stock_prices = stock_prices.filter(like='Adj Close')
stock_prices.columns = stock_prices.columns.get_level_values(1)

pairs = [('AMD', 'NVDA'), ('AMD', 'META'), ('NVDA', 'TSLA')]
spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = np.log(stock_prices[ticker_1]) - np.log(stock_prices[ticker_2])
    
    spread_mean = spread.expanding().mean()
    spread_std = spread.expanding().std()
    z_score = (spread - spread_mean) / spread_std

    position = np.where(z_score > 0, 'Sell', 'Buy')
    
    pair_df = pd.DataFrame({
        'Date': spread.index,
        'Ticker Pair': f'{ticker_1}-{ticker_2}',
        'Traditional Spread': spread.values,
        'Traditional Z-score': z_score.values,
        'Traditional Position': position
    })

    spread_list.append(pair_df)

full_traditional_df = pd.concat(spread_list)
full_traditional_df.to_csv("outputs/traditional_dummy2.csv", index=False)

[*********************100%***********************]  4 of 4 completed


In [74]:
df = pd.read_csv("traditional_dummy2.csv", parse_dates=["Date"])

returns = pd.read_csv("outputs/returns.csv", parse_dates=["Date"])
merged_df = df.merge(returns, on=["Date", "Ticker Pair"], how="left")

merged_df["Position"] = np.where(
    merged_df["Traditional Position"] == "Buy", 1,
    np.where(merged_df["Traditional Position"] == "Sell", -1, 0)
)

merged_df["Strategy Return"] = merged_df["Position"].shift(1) * merged_df["Return"]

merged_df["Strategy Net Return"] = merged_df["Strategy Return"] - 0.001 * np.abs(merged_df["Position"].diff().fillna(0))

merged_df.to_csv("outputs/traditional_results.csv", index=False)

In [76]:
spreads = pd.read_csv('outputs/spreads.csv', parse_dates=['Date'])
market_data = pd.read_csv('outputs/returns.csv', parse_dates=['Date'])
market_prices = market_data.pivot(index='Date', columns='Ticker', values='Adj Close')

In [78]:
pair_prices = market_prices[['AMD', 'NVDA']].dropna()
result = coint(pair_prices['AMD'], pair_prices['NVDA'])
print("P-value for cointegration test:", result[1])
print("Cointegrated?" , result[1] < 0.05)

P-value for cointegration test: 0.013859040978177854
Cointegrated? True


In [90]:
class TraditionalPairsTrading:
    def __init__(self, window=24, z_threshold=1.5):
        self.window = window
        self.z_threshold = z_threshold

    def calculate_features(self, df):
        df['Rolling_Mean'] = df.groupby('Ticker Pair')['Spread'].rolling(
            self.window, min_periods=int(self.window*0.8)).mean().values
        df['Rolling_Std'] = df.groupby('Ticker Pair')['Spread'].rolling(
            self.window, min_periods=int(self.window*0.8)).std().values
        df['Z_Score'] = (df['Spread'] - df['Rolling_Mean']) / df['Rolling_Std']
        return df.dropna()

    def generate_signals(self, df):
        df['Position'] = np.select(
            [df['Z_Score'] > self.z_threshold, df['Z_Score'] < -self.z_threshold],
            ['Sell', 'Buy'],
            default='Hold'
        )
        return df

    def backtest(self, df):
        df['Strategy_Return'] = df['Return'] * df['Position'].map({'Buy':1, 'Sell':-1, 'Hold':0})
        df['Cumulative_Return'] = (1 + df['Strategy_Return']).cumprod()
        return df

In [92]:
from sklearn.model_selection import ParameterGrid
import numpy as np

def calculate_sharpe_ratio(df, risk_free_rate=0.02):
    excess_returns = df['Strategy_Return'] - risk_free_rate / 252
    return np.sqrt(252) * excess_returns.mean() / excess_returns.std()

def calculate_max_drawdown(cum_returns):
    peak = cum_returns.cummax()
    drawdown = (cum_returns - peak) / peak
    return drawdown.min()

def optimize_parameters(train_data):
    param_grid = {
        'window': [12, 24, 36],
        'z_threshold': [1.0, 1.5, 2.0]
    }

    best_sharpe = -np.inf
    best_params = None

    for params in ParameterGrid(param_grid):
        strategy = TraditionalPairsTrading(**params)
        df_processed = strategy.calculate_features(train_data.copy())

        if df_processed.empty:
            continue

        df_with_signals = strategy.generate_signals(df_processed)
        df_backtest = strategy.backtest(df_with_signals)

        if df_backtest['Strategy_Return'].std() == 0:
            continue

        sharpe = calculate_sharpe_ratio(df_backtest)
        print(f"Tested {params} → Sharpe = {sharpe:.4f}")

        if np.isfinite(sharpe) and sharpe > best_sharpe:
            best_sharpe = sharpe
            best_params = params

    if best_params is None:
        print("⚠️ No valid parameter found. Using fallback default...")
        best_params = {'window': 24, 'z_threshold': 1.5}

    return best_params